In [15]:
!curl https://www.data.gouv.fr/fr/datasets/r/10bfaf29-2d13-48a3-bf6b-7ea8725c9ff2

<!doctype html>
<html lang=en>
<title>Redirecting...</title>
<h1>Redirecting...</h1>
<p>You should be redirected automatically to the target URL: <a href="https://static.data.gouv.fr/resources/etats-et-capitales-du-monde/20200526-142355/liste-197-etats-2020.csv">https://static.data.gouv.fr/resources/etats-et-capitales-du-monde/20200526-142355/liste-197-etats-2020.csv</a>. If not, click the link.


**Resources**
- https://github.com/SteinRobert/python-restcountries
- https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/
- https://fr.wikipedia.org/wiki/Listes_des_communes_de_France
- https://explore.data.gouv.fr/fr/datasets/53699502a3a729239d2045b5/#/resources/10bfaf29-2d13-48a3-bf6b-7ea8725c9ff2


**Autres**
- https://www.visamundi.io/fr/docs/endpoints-de-lapi/pays/liste-des-pays?utm_source=chatgpt.com
- https://iso.lahrim.fr/

In [1]:
# # https://www.data.gouv.fr/fr/datasets/r/10bfaf29-2d13-48a3-bf6b-7ea8725c9ff2


import pandas as pd

df_pays = pd.read_csv("liste-197-etats-2020.csv", encoding="latin-1", sep=";")

df_pays_filtre = df_pays[["NOM" , "CODE" , "CAPITALE"]]
df_pays_filtre.head()

,NOM,CODE,CAPITALE
0,Afghanistan,AFG,Kaboul
1,Afrique du Sud,ZAF,Prétoria
2,Albanie,ALB,Tirana
3,Algérie,DZA,Alger
4,Allemagne,DEU,Berlin


In [22]:
!pip install chardet


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
import chardet  # For automatic encoding detection

def safe_convert(text):
    if pd.isna(text):
        return text
    try:
        # First try UTF-8
        return str(text).encode('latin-1').decode('utf-8')
    except UnicodeError:
        try:
            # If UTF-8 fails, detect actual encoding
            encoding = chardet.detect(text.encode('latin-1'))['encoding']
            return text.encode('latin-1').decode(encoding or 'latin-1')
        except:
            return text  # Fallback to original if all fails

# Read with Latin-1 (will accept any byte sequence)
df_pays = pd.read_csv("liste-197-etats-2020.csv", encoding="latin-1", sep=";")

# Apply conversion only to string columns
str_cols = df_pays.select_dtypes(include=['object']).columns
for col in str_cols:
    df_pays[col] = df_pays[col].apply(safe_convert)

# Create filtered dataframe
df_pays_filtre = df_pays[["NOM", "CODE", "CAPITALE"]]
df_pays_filtre.head()

,NOM,CODE,CAPITALE
0,Afghanistan,AFG,Kaboul
1,Afrique du Sud,ZAF,Prétoria
2,Albanie,ALB,Tirana
3,Algérie,DZA,Alger
4,Allemagne,DEU,Berlin


In [4]:
OPENAI_API_KEY="sk-votre_clé_api",

In [7]:
import os
import openai
from openai import OpenAI
import pandas as pd
from pydantic import BaseModel
from typing import List

client = OpenAI(
    #api_key=os.environ.get("OPENAI_API_KEY"),
    api_key=OPENAI_API_KEY
)


In [12]:
df_pays

,NOM,CODE,CAPITALE
0,Afghanistan,AFG,Kaboul
1,Afrique du Sud,ZAF,Prétoria
2,Albanie,ALB,Tirana
3,Algérie,DZA,Alger
4,Allemagne,DEU,Berlin
...,...,...,...
192,Vénézuéla,VEN,Caracas
193,Vietnam,VNM,Hanoï
194,Yémen,YEM,Sanaa
195,Zambie,ZMB,Lusaka


In [8]:
import openai
import pandas as pd
import json
import time
from openai import OpenAI
from pydantic import BaseModel

# Initialize client
client = OpenAI(
    api_key=OPENAI_API_KEY
    #api_key="your-api-key"
)

class GentileResponse(BaseModel):
    gentile: str  # Using ASCII-friendly field name

def safe_encode(text):
    """Handle encoding conversion safely"""
    if pd.isna(text):
        return text
    try:
        return str(text).encode('latin-1', errors='ignore').decode('utf-8', errors='ignore')
    except:
        return text

def obtenir_gentile_struct(nom_pays):
    try:
        # Safely encode the prompt
        nom_pays_clean = safe_encode(nom_pays)
        prompt = f"Donne le gentilé du pays {nom_pays_clean}"
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Réponds avec le gentilé seulement"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=30,
            temperature=0
        )
        
        # Clean the response
        gentile = response.choices[0].message.content.strip()
        return safe_encode(gentile)
    
    except Exception as e:
        print(f"Error for {nom_pays}: {str(e)}")
        return None

# Read and prepare data
df_pays = pd.read_csv("liste-197-etats-2020.csv", encoding="latin-1", sep=";")
df_pays_filtre = df_pays[["NOM", "CODE", "CAPITALE"]].copy()

# Clean all string columns
for col in df_pays_filtre.select_dtypes(include=['object']):
    print("Starting with", df_pays_filtre[col])
    df_pays_filtre[col] = df_pays_filtre[col].apply(safe_encode)
    print("Ending with", df_pays_filtre[col])

# Get gentiles
df_pays_filtre['GENTILE'] = df_pays_filtre['NOM'].apply(obtenir_gentile_struct)

# Display results
print(df_pays_filtre.head())

Starting with 0         Afghanistan
1      Afrique du Sud
2             Albanie
3             Algérie
4           Allemagne
            ...      
192         Vénézuéla
193           Vietnam
194             Yémen
195            Zambie
196          Zimbabwé
Name: NOM, Length: 197, dtype: object
Ending with 0         Afghanistan
1      Afrique du Sud
2             Albanie
3              Algrie
4           Allemagne
            ...      
192            Vnzula
193           Vietnam
194              Ymen
195            Zambie
196           Zimbabw
Name: NOM, Length: 197, dtype: object
Starting with 0      AFG
1      ZAF
2      ALB
3      DZA
4      DEU
      ... 
192    VEN
193    VNM
194    YEM
195    ZMB
196    ZWE
Name: CODE, Length: 197, dtype: object
Ending with 0      AFG
1      ZAF
2      ALB
3      DZA
4      DEU
      ... 
192    VEN
193    VNM
194    YEM
195    ZMB
196    ZWE
Name: CODE, Length: 197, dtype: object
Starting with 0        Kaboul
1      Prétoria
2        Tirana
3     

In [9]:
df_pays_filtre

,NOM,CODE,CAPITALE,GENTILE
0,Afghanistan,AFG,Kaboul,Afghan
1,Afrique du Sud,ZAF,Prtoria,Sud-africain
2,Albanie,ALB,Tirana,Albanais
3,Algrie,DZA,Alger,Algrien
4,Allemagne,DEU,Berlin,Allemand(e)
...,...,...,...,...
192,Vnzula,VEN,Caracas,Venezuelien
193,Vietnam,VNM,Hano,Vietnamien
194,Ymen,YEM,Sanaa,Ymnite
195,Zambie,ZMB,Lusaka,Zambien(ne)


In [12]:
#!pip install openpyxl

In [13]:
#df_pays_filtre
df_pays_filtre.to_excel('liste-nationalite-fr.xlsx', index=False)

In [14]:
df_pays_filtre.to_csv('liste-nationalite-fr.csv', index=False)

In [16]:
df_pays_filtre.head(50)

,NOM,CODE,CAPITALE,GENTILE
0,Afghanistan,AFG,Kaboul,Afghan
1,Afrique du Sud,ZAF,Prtoria,Sud-africain
2,Albanie,ALB,Tirana,Albanais
3,Algrie,DZA,Alger,Algrien
4,Allemagne,DEU,Berlin,Allemand(e)
5,Andorre,AND,Andorre-la-Vieille,Andorran
6,Angola,AGO,Luanda,Angolais
7,Antigua-et-Barbuda,ATG,Saint John's,"Antiguais, Barbudans"
8,Arabie saoudite,SAU,Riyad,Saoudien
9,Argentine,ARG,Buenos Aires,Argentin
